### N-gram language models or how to write scientific papers

Мы обучим нашу языковую модель на статьх [ArXiv](http://arxiv.org/) и посмотрим, сможем ли мы сгенерировать похожую!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
#!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

,author,day,id,link,month,summary,tag,title,year
15853,"[{'name': 'Chenhui Chu'}, {'name': 'Sadao Kuro...",7,1606.02126v4,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",6,As alignment links are not given between Engli...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Supervised Syntax-based Alignment between Engl...,2016
2544,"[{'name': 'Marco Tulio Ribeiro'}, {'name': 'Sa...",16,1602.04938v3,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,"Despite widespread adoption, machine learning ...","[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...","""Why Should I Trust You?"": Explaining the Pred...",2016
30473,"[{'name': 'Ken Sakurada'}, {'name': 'Weimin Wa...",8,1712.02941v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",12,This paper presents a novel method for detecti...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Dense Optical Flow based Change Detection Netw...,2017
35383,"[{'name': 'Zengyou He'}, {'name': 'Xiaofei Xu'...",24,cs/0505060v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,The task of outlier detection is to find small...,"[{'term': 'cs.DB', 'scheme': 'http://arxiv.org...",A Unified Subspace Outlier Ensemble Framework ...,2005
16436,"[{'name': 'Bo Han'}, {'name': 'Will Radford'},...",20,1702.05821v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,Text generation is increasingly common but oft...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Post-edit Analysis of Collective Biography Gen...,2017


In [3]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'].replace("\n", ' '), axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Токенизация

Все как обычно, вы уже опытные. Данные кроме тебя никто не токенизирует. Займитесь очисткой данных. Используйте WordPunctTokenizer

In [4]:
# Task: convert lines (in-place) into strings of space-separated tokens. Import & use WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()
for i in range(len(lines)):
    lines[i] = ' '.join(tokenizer.tokenize(lines[i].lower()))

In [5]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-граммовая языковая модель

Языковая модель - это вероятностная модель, которая оценивает вероятность текста: совместную вероятность всех лексем $w_t$ в тексте $X$: $P(X) = P(w_1, \dots, w_T)$.

Это можно сделать, следуя правилу цепочки:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$

Проблема такого подхода заключается в том, что финальный термин $P(w_T \mid w_1, \dots, w_{T-1})$ зависит от $n-1$ предыдущих слов. Эту вероятность нецелесообразно оценивать для длинных текстов, например, $T = 1000$.

Одна из популярных аппроксимаций - предположить, что следующее слово зависит только от конечного количества предыдущих слов:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$.

Такая модель называется __n-gram language model__, где n - параметр. Например, в 3-граммовой языковой модели каждое слово зависит только от двух предыдущих слов.

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

Иногда такую аппроксимацию также можно встретить под названием _марковское предположение n-го порядка_.


Первым этапом построения такой модели является подсчет всех вхождений слов с учетом N-1 предыдущих слов

In [6]:
from tqdm import tqdm
from collections import defaultdict, Counter, deque

# special tokens:
# - `UNK` represents absent tokens,
# - `EOS` is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases:
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    for s in lines:
        sentence = s.split()
        for i in range(len(sentence)):
            prev = []
            for j in range(i - 1, i - n, -1):
                if j >= 0:
                    prev.append(sentence[j])
                else:
                    prev.append(UNK)
            counts[tuple(prev[::-1])][sentence[i]] += 1
        prev = []
        for j in range(len(sentence) - 1, len(sentence) - n, -1):
            if j >= 0:
                prev.append(sentence[j])
            else:
                prev.append(UNK)
        counts[tuple(prev[::-1])][EOS] += 1
    
    return counts


In [7]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [8]:
class NGramLanguageModel:
    def __init__(self, lines, n):
        """
        Train a simple count-based language model:
        compute probabilities P(w_t | prefix) given ngram counts

        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n

        counts = count_ngrams(lines, self.n)

        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)

        # populate self.probs with actual probabilities
        for pair in counts:
            denominator = sum(counts[pair].values())
            for i in counts[pair]:
                self.probs[pair][i] = counts[pair][i] / denominator

    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]

    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [9]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [11]:
lm = NGramLanguageModel(lines, n=3)

Процесс генерации последовательностей... ну, он последовательный. Вы храните список лексем и итеративно добавляете следующую лексему путем выборки с вероятностями.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Вместо выборки с вероятностями можно также попробовать всегда брать наиболее вероятную лексему, выборку среди топ-K наиболее вероятных лексем или выборку с температурой. В последнем случае (температура) выборка производится из

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$.

Где $\tau > 0$ - температура модели. Если $\tau << 1$, то более вероятные токены будут выбраны с еще большей вероятностью, а менее вероятные исчезнут.

In [12]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    d = lm.get_possible_next_tokens(prefix)
    d = dict(sorted(d.items(), key=lambda item: item[1]))
    words = list(d.keys())[::-1]
    probs = np.array(list(d.values()))[::-1]
    if temperature == 0:
        return words[0]
    denom = (probs ** (1 / temperature)).sum()
    probs = probs ** (1 / temperature) / denom
    q = np.random.rand()
    i = 0
    while q > 0:
        q -= probs[i]
        i += 1
    return words[i - 1]


In [13]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


In [14]:
prefix = 'native language processing' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

native language processing tools . the computational structure w . r . t . the main contribution of a word may actually be achieved , compared to other algorithms by rederiving them in terms of both the link between microscopic and phenomenological models . in this paper is the weakest tongue depending on the caltech pedestrians dataset show that the estimated value function , and the attribution performances obtained from the domains . for quaternions ). already in the corresponding map estimation procedure called gibbs sampling , an application in natural images , providing distributed versions of a digital head phantom obtained with


In [15]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

bridging the gap between the two - sided probing , i . e ., a particular class of models and is capable of handling a large set of binary concept learning in a variety of natural language processing ( nlp ) tasks in environments when many types of annotations for training the network . this paper , we propose a novel approach for capturing the image is a challenging task due to the data . we also consider the problem of estimating the probability of a field - of - the - art approaches . we demonstrate that the proposed method is


__Больше в лабе:__ nucleus sampling, top-k sampling, beam search(не для слабонервных).

### Оценка языковых моделей: perplexity (1 балл)

_perplexity - это мера того, насколько хорошо ваша модель аппроксимирует истинное распределение вероятностей, лежащее в основе данных. Меньшая perplexity = лучше модель_.

Чтобы вычислить недоумение на одном предложении, используйте:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


На уровне корпуса текстов perplexity - это произведение вероятностей всех лексем во всех предложениях в степени $1/N$, где $N$ - _общая длина (в лексемах) всех предложений в корпусе текстов_.

Это число может быстро стать слишком маленьким для точности float32/float64, поэтому мы рекомендуем сначала вычислить log-perplexity (из log-probabilities), а затем взять экспоненту.

In [16]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above

    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)

    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    n = lm.n
    final_prob = 0
    len_s = 0
    for s in lines:
        sentence = s.split()
        sum_prob = 0
        for i in range(len(sentence)):
            if i == 0:
                prob = lm.get_next_token_prob(' '.join([UNK] * n), sentence[i])
                sum_prob += max(np.log(prob), min_logprob) if prob > 0 else min_logprob
            elif i < n:
                prob = lm.get_next_token_prob(' '.join([UNK] * (n - i) + sentence[0:i]), sentence[i])
                sum_prob += max(np.log(prob), min_logprob) if prob > 0 else min_logprob
            else:
                prob = lm.get_next_token_prob(' '.join(sentence[i - n:i]), sentence[i])
                sum_prob += max(np.log(prob), min_logprob) if prob > 0 else min_logprob
        i = len(sentence)
        if i < n:
            prob = lm.get_next_token_prob(' '.join([UNK] * (n - i) + sentence[0:i]), EOS)
            sum_prob += max(np.log(prob), min_logprob) if prob > 0 else min_logprob
        else:
            prob = lm.get_next_token_prob(' '.join(sentence[i - n:i]), EOS)
            sum_prob += max(np.log(prob), min_logprob) if prob > 0 else min_logprob
        final_prob += sum_prob
        len_s += len(sentence) + 1
    return np.exp((-final_prob) / len_s)

In [18]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be non-negative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


Давайте теперь измерим perplexity по факту:

In [19]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


N = 1, Perplexity = 1832.23136
N = 2, Perplexity = 85653987.28543
N = 3, Perplexity = 61999196239911532363776.00000


In [ ]:
# whoops, it just blew up :)

### LM Smoothing

Проблема нашей простой языковой модели заключается в том, что всякий раз, когда она встречает n-грамму, которую никогда раньше не видела, она присваивает ей вероятность 0. Каждый раз, когда это происходит, недоумение взрывается.

Для борьбы с этой проблемой существует техника, называемая __сглаживанием__. Суть ее заключается в том, чтобы изменить подсчеты таким образом, чтобы не допустить слишком низкого значения вероятности. Простейшим алгоритмом здесь является аддитивное сглаживание (оно же [сглаживание Лапаса](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

Если количество префиксов невелико, аддитивное сглаживание приведет вероятности к более равномерному распределению. Не стоит забывать, что суммирование в знаменателе идет по _всем словам в словаре_.

In [20]:
class LaplaceLanguageModel(NGramLanguageModel):
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}

    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))


In [21]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [22]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 1832.66878
N = 2, Perplexity = 470.48021
N = 3, Perplexity = 3679.44765


In [ ]:
# optional: try to sample tokens from such a model

### Сглаживание Кнезера-Нея (4 балла)

Сглаживание Лапласа - простой, достаточно хороший метод, но точно не SOTA


Ваша последняя задача в этом ноутбуке - реализовать сглаживание [Кнезера-Нея](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing).

Оно может быть вычислено рекуррентно, для n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

где
- $prefix_{n-1}$ - кортеж из {n-1} предыдущих лексем
- $lambda_{prefix_{n-1}}$ - константа нормализации, выбранная таким образом, чтобы вероятности складывались в 1
- Униграмма $P_{kn}(w_t | prefix_{n-2})$ соответствует сглаживанию Кнезера-Нея для {N-1}-грамматической языковой модели.
- Униграмма $P_{kn}(w_t)$ - это частный случай: насколько вероятно увидеть x_t в незнакомом контексте.

Более подробные формулы см. на слайдах лекции или в вики.

__Ваша задача__ состоит в том, чтобы
- реализовать класс `KneserNeyLanguageModel`,
- протестировать его на 1-3 грамматических языковых моделях
- найти оптимальную (в пределах разумного) дельту сглаживания для 3-граммовой языковой модели со сглаживанием Кнесер-Нея

In [32]:
class KneserNeyLanguageModel(NGramLanguageModel):
    """ A template for Kneser-Ney language model. Default delta may be suboptimal. """
    def __init__(self, lines, n, delta=1.0):
        self.lines = lines
        self.n = n
        self.delta = delta
        self.counts = dict()
        for index in range(1, self.n + 1):
            self.counts[index] = count_ngrams(lines, index)
        self.vocab = set(token for token_counts in self.counts[self.n].values() for token in token_counts)
        self.probs = self.evaluate_probs(self.n)

    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}

    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        
        if next_token in token_probs:
            return token_probs[next_token]

        missing_prob_total = max(0, 1.0 - sum(token_probs.values()))
        return missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        
    def evaluate_probs(self, n):
        if n == 1:
            probs = defaultdict(Counter)
            for prefix in self.counts[1]:
                token_counts = self.counts[1][prefix]
                total_count = sum(token_counts.values())
                lambda_ = self.delta * len(self.vocab) / total_count
                probs[prefix] = {token: max(0, token_counts[token] - self.delta) / total_count + lambda_ / len(self.vocab) for token in token_counts}
            return probs
            
        probs = self.evaluate_probs(n-1)
        for prefix in self.counts[n]:
            token_counts = self.counts[n][prefix]
            total_count = sum(token_counts.values())
            lambda_ = 0 if n == self.n else self.delta * len(token_counts) / total_count
            prev_probs = probs[prefix]
            sec = {token: lambda_ * prev_probs[token] for prefix in self.counts[n - 1] for token in token_counts}
            probs[prefix] = {token: max(0, token_counts[token] - self.delta) / total_count + sec[token] for token in token_counts}
        return probs


In [33]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [34]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n, delta=1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 1832.23136
N = 2, Perplexity = 26647.18030
